# Table of Contents
1. Creating UCCSD ansatz circuit for LiH
2. Optimizing circuit
3. Mapping circuit to nearest neighbor graph
-----
## 1. Creating UCCSD Ansatz Circuit for LiH

In [1]:
import sys
sys.path.append('..')
from fqc import uccsd, util
import numpy as np

In [2]:
theta_vector = [np.random.rand() * 2 * np.pi for _ in range(8)]
print(theta_vector)

[3.181891975375853, 1.3747357738804031, 1.7184949513258354, 4.12424274938467, 0.26206104280255, 5.725221176817354, 1.1781097705813257, 0.9911897316089741]


In [3]:
circuit = uccsd.get_uccsd_circuit('LiH', theta_vector)
circuit.draw(output='text', line_length=4000)

In [4]:
util.circuitutil.get_max_pulse_time(circuit)

1501.699999999997

In [5]:
unoptimized_unitary = util.get_unitary(circuit)

## 2. Optimizing circuit

In [6]:
circuit = util.circuitutil.optimize_circuit(circuit)

In [7]:
circuit.draw(output='text', line_length=4000)

In [8]:
util.circuitutil.get_max_pulse_time(circuit)

1238.1999999999994

In [9]:
optimized_unitary = util.get_unitary(circuit)
print(np.array_equal(optimized_unitary * -1, unoptimized_unitary))  # (equal up to -1 global phase)

True


## 3. Mapping circuit to nearest neighbor graph

In [10]:
coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2)

In [11]:
mapped_circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)

In [12]:
mapped_circuit.draw(output='text', line_length=4000)

In [13]:
util.circuitutil.get_max_pulse_time(mapped_circuit)

1315.399999999998

In [14]:
mapped_unitary = util.get_unitary(mapped_circuit)
mapped_unitary[1] - unoptimized_unitary[1]

matrix([[-0.00243504-0.00221163j, -0.06039121-0.05485031j,
          0.06044028+0.05489488j,  0.        +0.j        ,
         -0.01636651-0.01486488j, -0.40590378-0.36866206j,
          0.40623361+0.36896162j,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
          0.        +0.j        ,  0.        +0.j        ,
         -0.02481227-0.02253574j, -0.61536612-0.5589062j ,
          0.61586615+0.55936035j,  0.        +0.j        ]])

<font color='red'>^^^These two unitaries should be the same (up to global phase), but for example, these columns differ.</font>

In [ ]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [ ]:
import numpy as np
import os, sys, math
from datetime import datetime
sys.path.append('..'); import config

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [ ]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 1000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}

In [ ]:
N = 4
connected_qubit_pairs = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = hamiltonian.get_H0(N, d, connected_qubit_pairs)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d)
U = transmon_gate(optimized_unitary, d)

In [ ]:
total_time = 50.3
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

In [ ]:
# allow 2000 iterations-->total_time is much smaller :(

d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay}
reg_coeffs = {}
total_time = 44.4
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)